In [1]:
import sys
sys.path.append("..")

import os

In [22]:
import subprocess
subprocess.run( ["sirius", "--help"] )

Dec 13, 2024 4:09:45 PM org.apache.commons.beanutils.FluentPropertyBeanIntrospector introspect
INFO: Error when creating PropertyDescriptor for public final void org.apache.commons.configuration2.AbstractConfiguration.setProperty(java.lang.String,java.lang.Object)! Ignoring this property.


Usage: sirius [-hV] [--noCite] [--recompute] [--buffer=<initialInstanceBuffer>]
              [--cores=<numOfCores>] [--log=<logLevel>] [--maxmz=<maxMz>]
              [--workspace=<workspace>] [[-o=<outputProjectLocation>]
              [--update-fingerprint-version]] [[-i=<inputPath>[,<inputPath>...]
              [-i=<inputPath>[,<inputPath>...]]... [--ignore-formula]
              [--allow-ms1-only]] [-z=<parentMz> [-f=<formula>] [-1=<ms1File>[,
              <ms1File>...]] -2=<ms2File>[,<ms2File>...]
              [--adduct=<ionType>]]...] [COMMAND]
  -h, --help                Show this help message and exit.
  -V, --version             Print version information and exit.
      --log, --loglevel=<logLevel>
                            Set logging level of the Jobs SIRIUS will execute.
                              Valid values: SEVERE, WARNING, INFO, FINER, ALL
                              Default: WARNING
      --cores, --threads, --processors=<numOfCores>
                       

Dec 13, 2024 4:09:46 PM de.unijena.bioinf.ChemistryBase.jobs.SiriusJobs getGlobalJobManager
INFO: Job manager successful initialized with 15 CPU thread(s) and 2 IO thread(s).


CompletedProcess(args=['sirius', '--help'], returncode=0)